## Evaluate Progress of Labellers

Use this notebook to evaluate the progress of labellers and to inspect their labels against imagery. This notebook requires python 3. 

## Set-up 

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('src'))
sys.path.insert(0, module_path)

from labelreview import labelReview

## Get assignment data

In [ ]:
lr = labelReview(config="config-db.yaml")
query = \
    "SELECT name,hit_id,assignment_id,worker_id,email,score,status,kml_type "\
    "FROM assignment_data "\
    "LEFT JOIN hit_data USING (hit_id) "\
    "LEFT JOIN kml_data USING (name) "\
    "LEFT JOIN users ON assignment_data.worker_id = users.id"
assignments = lr.get_data(query)

### Summarize assignment counts and score

In [ ]:
counts = assignments[["worker_id", "email", "kml_type"]]\
    .groupby(["worker_id", "email", "kml_type"], as_index=False)\
    .value_counts()

scores = assignments[["worker_id", "score"]]\
    .groupby("worker_id", as_index=False)\
    .mean("score")

In [ ]:
counts.pivot_table(values="count", index=["worker_id", "email"], 
                   columns="kml_type")

In [ ]:
counts.pivot_table(values="count", index="worker_id", columns="kml_type")\
    .plot(subplots=True, kind="bar")
None

In [ ]:
scores

In [ ]:
# assignments.query("worker_id==13 & kml_type=='Q'")["score"]
assignments.query("kml_type=='Q'")[["worker_id", "score"]]\
    .boxplot(by='worker_id', column='score', grid=False)
None

In [ ]:
# assignments.query("worker_id==20 & kml_type=='Q'")["score"]

## Compare maps

Select a particular labeller's map by ID, and then evaluate their labels at a particular site. 

Labels can be selected as follows:

- By type of task, F or Q. If you select "Q" the labeller's maps will be shown against the expert maps for the same site.
- Through random choice, or for a particular site name.

The arguments in the function `get_labels` enable these choices. 

For example, the following call will get one randomly selected Q type site for labeller 10. 

```python
labels = get_labels(id=10)
```
This one will get a site by a particular name:

```python
labels = get_labels(id=13, name="ET0472958")
```


In [ ]:
labels = lr.get_labels(assignments, id=13)
# labels = get_labels(id=13, name="ET0472958")

### Map

In [ ]:
lr.plot_labels(labels)